In [ ]:
import roslib
import rosbag
import rospy
import cv2
import datetime
import numpy as np
import ros_numpy
import os
from sensor_msgs.msg import PointCloud2, PointField, Image
from datetime import *
from cv_bridge import CvBridge
from cv_bridge import CvBridgeError



In [ ]:
rosbag_path = '/home/user/catkin_ws/yolov5/data/rosbag/'
rosbag_file = os.path.join(rosbag_path, '2022-10-21-13-59-30.bag')

image_path = os.path.join(rosbag_path, 'imgages')
image_Time_Stamps_File_path = os.path.join(image_path, 'timestamps.txt')
pointcloud_path =  os.path.join(rosbag_path, 'velodyne_points')
pointcloud_Time_Stamps_File_path = os.path.join(pointcloud_path, 'timestamps.txt')

image_topic_name = '/hikrobot_camera/image_raw'
pointcloud_topic_name = '/velodyne_points'


def ImageCreator():
    bridge = CvBridge()
    timeStampsFile = open(image_Time_Stamps_File_path, 'w')
    with rosbag.Bag(rosbag_file, 'r') as bag:
        num = 0
        for topic, msg, t in bag.read_messages():
            if topic == image_topic_name:
                try:
                    cv_image = bridge.imgmsg_to_cv2(msg, "bgr8")
                except CvBridgeError as e:
                    print(e)

                pointcloud_name = "%010d" % num + ".png"
                cv2.imwrite(image_path + pointcloud_name, cv_image)

                timeStr = "%.9f" % msg.header.stamp.to_sec()
                dateTimeStr = int(timeStr[0:10])
                dotTimeStr = timeStr[11:]

                localTime = datetime.fromtimestamp(dateTimeStr)
                localTimeStr = str(localTime)
                timeStamps = localTimeStr + '.' + dotTimeStr

                timeStampsFile.write(timeStamps)
                timeStampsFile.write('\n')
                num += 1


def PointcloudCreator():
    timeStampsFile = open(pointcloud_Time_Stamps_File_path, 'w')
    with rosbag.Bag(rosbag_file, 'r') as bag:
        num = 0
        for topic, msg, t in bag.read_messages():
            if topic == pointcloud_topic_name:
                PointCloud = msg.data
                pointcloud_name = "%010d" % num
                PointCloudFile = open(os.path.join(pointcloud_path, pointcloud_name), 'wb')

                xyz_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(msg)
                val = np.save(PointCloudFile, xyz_array)
                PointCloudFile.close()

                timeStr = "%.9f" % msg.header.stamp.to_sec()
                dateTimeStr = int(timeStr[0:10])
                dotTimeStr = timeStr[11:]

                localTimeStr = datetime.fromtimestamp(dateTimeStr)
                localTimeStr = str(localTimeStr)
                timeStamps = localTimeStr + '.' + dotTimeStr

                timeStampsFile.write(timeStamps)
                timeStampsFile.write('\n')
                num += 1

def eval_timestamps():
    #open two files and compare timestamps
    cam_stamps = open(os.path.join(image_path, 'timestamps.txt'), 'r')
    ldr_stamps = open(os.path.join(pointcloud_path, 'timestamps.txt'), 'r')

In [ ]:
image_creator = ImageCreator()

In [ ]:
pc_creator =PointcloudCreator()